## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888

Sunny, Rainy, Sunny: 0.4*(0.6)*0.4*(0.1)*0.3*(0.6) = 0.001728
Rainy, Rainy, Sunny: 0.6*(0.1)*0.7*(0.1)*0.3*(0.6) = 0.000756

Sunny, Sunny, Rainy: 0.4*(0.6)*0.6*(0.6)*0.4*(0.1) = 0.003456
Rainy, Sunny, Rainy: 0.6*(0.1)*0.3*(0.6)*0.4*(0.1) = 0.000432


Sunny, Rainy, Rainy: 0.4*(0.6)*0.4*(0.1)*0.7*(0.1) = 0.000672
Rainy, Rainy, Rainy: 0.6*(0.1)*0.7*(0.1)*0.7*(0.1) = 0.000294

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [19]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
observations1 = ('damp', 'damp', 'dry')
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [10]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}
    
    # Initialize base cases (t == 0)
    for y in states:                                             # t = 0, 狀態為某 state 的機率
        V[0][y] = start_p[y]*emit_p[y][obs[0]]                   # 某狀態下的機率 = 初始機率 * 觀察水草所得發射矩陣中機率
        path[y] = [y]                                            # 紀錄 t = 0 的狀態路徑
    
    print(V)
    # Run Viterbi for t > 0
    for t in range(1,len(obs)):                                   # 到時間 t, 狀態為某 state 的機率
        v = {}
        newpath = {}
        for y in states:                                          # 比較從 t-1 的各狀態 x 轉移到時間 t 狀態為某狀態 y的機率
            mx = 0                                                # 取最大值作為 : 到時間 t, 該狀態 y 的機率
            for x in states:                                      # 並記錄新的狀態路徑
                m = V[t-1][x]*trans_p[x][y]*emit_p[y][obs[t]]
                if m > mx:
                    mx = m
                    pre_state = path[x]
            v[y] = mx
            newpath[y] = pre_state + [y]
        
        V.append(v)
        # Don't need to remember the old paths
        path = newpath
    
    print(V)
    print(path)
    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [11]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

[{'sunny': 0.24, 'rainy': 0.06}]
[{'sunny': 0.24, 'rainy': 0.06}, {'sunny': 0.08639999999999999, 'rainy': 0.009600000000000001}, {'sunny': 0.031103999999999993, 'rainy': 0.0034560000000000003}]
{'sunny': ['sunny', 'sunny', 'sunny'], 'rainy': ['sunny', 'sunny', 'rainy']}


In [12]:
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])

In [20]:
result1 = viterbi(observations1,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

[{'sunny': 0.12, 'rainy': 0.24}]
[{'sunny': 0.12, 'rainy': 0.24}, {'sunny': 0.021599999999999998, 'rainy': 0.0672}, {'sunny': 0.012095999999999997, 'rainy': 0.004704}]
{'sunny': ['rainy', 'rainy', 'sunny'], 'rainy': ['rainy', 'rainy', 'rainy']}


In [21]:
result1

(0.012095999999999997, ['rainy', 'rainy', 'sunny'])